In [1]:
%load_ext autoreload

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torchvision.datasets as datasets
import torchvision
from torch.utils.data import DataLoader
from pathlib import Path
import sys
sys.path.append(str(Path('../../').resolve()))

In [3]:
import torch.optim as optim
optim.lr_scheduler._LRScheduler

torch.optim.lr_scheduler._LRScheduler

In [4]:
%autoreload
import v4.pytorch_unet.unet as unet
from v4.pytorch_unet.learner import LearnerCallback, UNetLearner
from v4.pytorch_unet.unet_dataset import UNetDataset

In [5]:
train_dataset = UNetDataset(root_dir=Path('simple_dataset'), part='train')
valid_dataset = UNetDataset(root_dir=Path('simple_dataset'), part='valid')
train_loader = DataLoader(train_dataset)
valid_loader = DataLoader(valid_dataset)

In [6]:
model = unet.ResNetUNet(in_channels=train_dataset.image_channels, in_size=train_dataset.item_size, n_classes=len(train_dataset.classes), depth=4)

In [7]:
def handle_epoch_end(metrics, epoch):
    learner.save_checkpoint(Path('test_checkpoint'), 'some_model')
    
callback = LearnerCallback(epoch_end=handle_epoch_end)

learner = UNetLearner(model, train_loader, valid_loader, cuda=True, callback=callback)

In [8]:
train_dataset[0][1].shape

torch.Size([300, 300])

In [ ]:
learner.train(n_epochs=10, lr=3e-4, momentum=0.9)

Epoch 5
train_loss 0.3109031011660894
valid_loss 0.3089006543159485
train_accuracy 0.3739685185185184
valid_accuracy 0.3707055555555555
------------------
learning rate: [0.00018337860658088678]


In [ ]:
{key: 1 for (key, value) in learner.epoch_metrics.items()}

In [11]:
test = {'train_loss': 0, 'valid_loss': 0, 'accuracy': 0}
{key: value for key, value in enumerate(test)}

{0: 'train_loss', 1: 'valid_loss', 2: 'accuracy'}

In [14]:
plt.ioff()
figure = learner.show_train_results(2)
figure.savefig("test.png")
plt.ion()